<a href="https://colab.research.google.com/github/seabnavin19/Cambodia-Football-Match-result-prediction/blob/main/football_match_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data installation

In [42]:
#change current dir
import os
os.environ["KAGGLE_CONFIG_DIR"]="/content/drive/MyDrive/kaggle"

In [3]:
%cd /content/drive/MyDrive/kaggle

/content/drive/MyDrive/kaggle


In [ ]:
!ls

AAPL.csv  kaggle.json	      nexflex	    Rice_Final_Dataset
churn	  netflix_titles.csv  rice_disease  tes


In [ ]:
!kaggle datasets download -d martj42/international-football-results-from-1872-to-2017

  0% 0.00/549k [00:00<?, ?B/s]
100% 549k/549k [00:00<00:00, 17.5MB/s]


In [ ]:
!mkdir footballMatch

In [ ]:
!mv international-football-results-from-1872-to-2017.zip footballMatch

In [ ]:
%cd footballMatch/

/content/drive/My Drive/kaggle/footballMatch


In [ ]:
!ls

international-football-results-from-1872-to-2017.zip


In [ ]:
!unzip international-football-results-from-1872-to-2017.zip

Archive:  international-football-results-from-1872-to-2017.zip
  inflating: results.csv             
  inflating: shootouts.csv           


In [ ]:
!ls

international-football-results-from-1872-to-2017.zip  shootouts.csv
results.csv


## Data loading and Exploration


In [43]:
df=pd.read_csv("/content/drive/MyDrive/kaggle/footballMatch/shootouts.csv")
df.head()

,date,home_team,away_team,winner
0,1967-08-22,India,Taiwan,Taiwan
1,1971-11-14,South Korea,Vietnam Republic,South Korea
2,1972-05-17,Thailand,South Korea,South Korea
3,1972-05-19,Thailand,Cambodia,Thailand
4,1973-07-26,Malaysia,Kuwait,Malaysia


In [44]:
df_re=pd.read_csv("/content/drive/MyDrive/kaggle/footballMatch/results.csv")
df_re.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [45]:
df_re[(df_re["home_team"]=="Cambodia") | (df_re["away_team"]=="Cambodia")]

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
3731,1954-11-01,Cambodia,Thailand,7,1,Friendly,Phnom Penh,Cambodia,False
3940,1956-03-17,Malaysia,Cambodia,9,2,AFC Asian Cup qualification,Kuala Lumpur,Malaya,True
3955,1956-04-24,Cambodia,Malaysia,3,2,AFC Asian Cup qualification,Phnom Penh,Cambodia,False
4228,1957-08-31,Cambodia,Hong Kong,2,6,Merdeka Tournament,Kuala Lumpur,Malaya,True
4231,1957-09-01,Cambodia,Thailand,0,3,Merdeka Tournament,Kuala Lumpur,Malaya,True
...,...,...,...,...,...,...,...,...,...
41469,2019-11-14,Cambodia,Mongolia,1,1,Friendly,Phnom Penh,Cambodia,False
41605,2019-11-19,Hong Kong,Cambodia,2,0,FIFA World Cup qualification,Hong Kong,Hong Kong,False
42170,2021-06-03,Bahrain,Cambodia,8,0,FIFA World Cup qualification,Riffa,Bahrain,False
42238,2021-06-07,Iraq,Cambodia,4,1,FIFA World Cup qualification,Arad,Iran,True


In [46]:
## prepare a new dataset which is remove cambodia because we want to predict cambodia result so cambodia not a feature anymore

# when cambodia is home team
df_home=df_re[df_re['home_team']=="Cambodia"]
me=df_home["home_score"].values-df_home["away_score"].values
df_home=df_home[["away_team","tournament","country","neutral"]]
df_home.rename(columns={"away_team":"team"},inplace=True)
df_home["score"]=me

#when cambodia is away team
df_away=df_re[df_re['away_team']=="Cambodia"]
me=df_away["away_score"].values-df_away["home_score"].values
df_away=df_away[["home_team","tournament","country","neutral"]]
df_away["score"]=me

df_away.rename(columns={"home_team":"team"},inplace=True)
df_away.head()

,team,tournament,country,neutral,score
3940,Malaysia,AFC Asian Cup qualification,Malaya,True,-7
4237,Burma,Merdeka Tournament,Malaya,True,-2
5941,North Korea,Friendly,North Korea,False,-4
6678,Burma,AFC Asian Cup qualification,Burma,False,-1
7645,Indonesia,King's Cup,Thailand,True,-3


In [47]:
df_cam=df_home.append(df_away)

## Data Prepeocessing

In [48]:
df_cam.head()

,team,tournament,country,neutral,score
3731,Thailand,Friendly,Cambodia,False,6
3955,Malaysia,AFC Asian Cup qualification,Cambodia,False,1
4228,Hong Kong,Merdeka Tournament,Malaya,True,-4
4231,Thailand,Merdeka Tournament,Malaya,True,-3
4245,Singapore,Merdeka Tournament,Malaya,True,0


In [49]:
# for tournament i devide it into 2 types friendly and challenge
df_cam.loc[df_cam["tournament"]!="Friendly","tournament"]="others"
df_cam.loc[df_cam["score"]>0,"result"]="win"
df_cam.loc[df_cam["score"]<0,"result"]="lose"
df_cam.loc[df_cam["score"]==0,"result"]="draw"

In [50]:
df_cam.loc[df_cam["country"]=="Cambodia","home"]="True"
df_cam.loc[df_cam["country"]!="Cambodia","home"]="False"


In [51]:
df_cam.drop(["country"],axis=1,inplace=True)

In [52]:
from collections import Counter
list(Counter(df_cam["result"]).keys())

['win', 'lose', 'draw']

In [53]:
percent_win_friend=df_cam[df_cam["tournament"]=="Friendly"]["result"].value_counts()["win"]/df.shape[0]
percent_win_challenge=df_cam[df_cam["tournament"]=="others"]["result"].value_counts()["win"]/df.shape[0]
print("percentage win in friendly match:"+str(percent_win_friend))
print("percentage win in formal match:"+str(percent_win_challenge))

percentage win in friendly match:0.018648018648018648
percentage win in formal match:0.09324009324009325


In [57]:
# Data preprocessing
from sklearn.preprocessing import LabelEncoder

# lb_make = LabelEncoder()
# df_cam["team"] = lb_make.fit_transform(df_cam["team"])
# df_cam.head()

df_team=pd.get_dummies(df_cam["team"])
df_cam1 = pd.concat([df_cam,df_team],axis='columns')
# in dummies variable you need to drop the old variable and one label
df_cam1.drop(["team","Afghanistan"],axis=1,inplace=True)
df_cam1.head()

df_tour=pd.get_dummies(df_cam["tournament"])
df_cam2 = pd.concat([df_cam1,df_tour],axis='columns')
# in dummies variable you need to drop the old variable and one label
df_cam2.drop(["tournament","others"],axis=1,inplace=True)
df_cam2.head()




,neutral,score,result,home,Australia,Bahrain,Bangladesh,Bhutan,Brunei,Burma,China PR,East Timor,Guam,Hong Kong,India,Indonesia,Iran,Iraq,Japan,Jordan,Kuwait,Kyrgyzstan,Laos,Macau,Malaysia,Maldives,Mongolia,Myanmar,Nepal,North Korea,North Vietnam,Pakistan,Palestine,Philippines,Saudi Arabia,Singapore,South Korea,Sri Lanka,Syria,Taiwan,Tajikistan,Thailand,Timor-Leste,Turkmenistan,Uzbekistan,Vietnam,Vietnam Republic,Western Australia,Yemen,Friendly
3731,False,6,win,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3955,False,1,win,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4228,True,-4,lose,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4231,True,-3,lose,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4245,True,0,draw,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
df_cam2["neutral"].replace({False:0,True:1},inplace=True)
df_cam2["home"].replace({"False":0,"True":1},inplace=True)
df_cam2.head()

,neutral,score,result,home,Australia,Bahrain,Bangladesh,Bhutan,Brunei,Burma,China PR,East Timor,Guam,Hong Kong,India,Indonesia,Iran,Iraq,Japan,Jordan,Kuwait,Kyrgyzstan,Laos,Macau,Malaysia,Maldives,Mongolia,Myanmar,Nepal,North Korea,North Vietnam,Pakistan,Palestine,Philippines,Saudi Arabia,Singapore,South Korea,Sri Lanka,Syria,Taiwan,Tajikistan,Thailand,Timor-Leste,Turkmenistan,Uzbekistan,Vietnam,Vietnam Republic,Western Australia,Yemen,Friendly
3731,0,6,win,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3955,0,1,win,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4228,1,-4,lose,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4231,1,-3,lose,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4245,1,0,draw,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
X=df_cam2.drop("result",axis=1)
Y=df_cam["result"]
X.head()

,neutral,score,home,Australia,Bahrain,Bangladesh,Bhutan,Brunei,Burma,China PR,East Timor,Guam,Hong Kong,India,Indonesia,Iran,Iraq,Japan,Jordan,Kuwait,Kyrgyzstan,Laos,Macau,Malaysia,Maldives,Mongolia,Myanmar,Nepal,North Korea,North Vietnam,Pakistan,Palestine,Philippines,Saudi Arabia,Singapore,South Korea,Sri Lanka,Syria,Taiwan,Tajikistan,Thailand,Timor-Leste,Turkmenistan,Uzbekistan,Vietnam,Vietnam Republic,Western Australia,Yemen,Friendly
3731,0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3955,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4228,1,-4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4231,1,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4245,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
Y.shape,X.shape

((228,), (228, 49))

In [71]:
Y = np.where(Y=="win", 0, Y)
Y = np.where(Y=="lose", 1, Y)
Y = np.where(Y=="draw", 2, Y)

In [92]:
Counter(Y)
Y=Y.astype('int')

In [93]:

## RandomOverSampler to handle imbalanced data

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
strategy = {0:149, 1:149, 2:149}
oversample = SMOTE(sampling_strategy=strategy)
X_res, Y_res = oversample.fit_resample(X, Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [78]:
X_res

array([[ 0,  6,  1, ...,  0,  0,  1],
       [ 0,  1,  1, ...,  0,  0,  0],
       [ 1, -4,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  0,  0,  0]])

In [94]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_res,Y_res,test_size=0.3)

In [95]:
X_train.shape,Y_train.shape

((312, 49), (312,))

In [96]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [107]:
model=GaussianNB()
model.fit(X_train,Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [108]:
model.score(X_test,Y_test)

0.5333333333333333

In [100]:
model1=RandomForestClassifier()
model1.fit(X_train,Y_train)
model1.score(X_test,Y_test)

1.0

In [103]:
model1.score(X_test,Y_test)

1.0

In [105]:
model1.predict([X_test[1]])

array([1])

In [106]:
model2=LogisticRegression()
model2.fit(X_train,Y_train)
model2.score(X_test,Y_test)

1.0